In [ ]:
import os
import cv2
import glob
import h5py
from scipy.io import loadmat
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def gen_density_map_gaussian(im, points, sigma=4):
    """
    func: generate the density map
    """
    density_map = np.zeros(im.shape[:2], dtype=np.float32)
    h, w = density_map.shape[:2]
    num_gt = np.squeeze(points).shape[0]
    if num_gt == 0:
        return density_map
    if sigma == 4:
        # Adaptive sigma in CSRNet.
        leafsize = 2048
        tree = scipy.spatial.KDTree(points.copy(), leafsize=leafsize)
        distances, _ = tree.query(points, k=4)
    for idx_p, p in enumerate(points):
        p = np.round(p).astype(int)
        p[0], p[1] = min(h-1, p[1]), min(w-1, p[0])
        gaussian_radius = sigma * 2 - 1
        if sigma == 4:
            # Adaptive sigma in CSRNet.
            sigma = max(int(np.sum(distances[idx_p][1:4]) * 0.1), 1)
            gaussian_radius = sigma * 3
        gaussian_map = np.multiply(
            cv2.getGaussianKernel(int(gaussian_radius*2+1), sigma),
            cv2.getGaussianKernel(int(gaussian_radius*2+1), sigma).T
        )
        x_left, x_right, y_up, y_down = 0, gaussian_map.shape[1], 0, gaussian_map.shape[0]
        # cut the gaussian kernel
        if p[1] < gaussian_radius:
            x_left = gaussian_radius - p[1]
        if p[0] < gaussian_radius:
            y_up = gaussian_radius - p[0]
        if p[1] + gaussian_radius >= w:
            x_right = gaussian_map.shape[1] - (gaussian_radius + p[1] - w) - 1
        if p[0] + gaussian_radius >= h:
            y_down = gaussian_map.shape[0] - (gaussian_radius + p[0] - h) - 1
        gaussian_map = gaussian_map[y_up:y_down, x_left:x_right]
        if np.sum(gaussian_map):
            gaussian_map = gaussian_map / np.sum(gaussian_map)
        density_map[
            max(0, p[0]-gaussian_radius):min(h, p[0]+gaussian_radius+1),
            max(0, p[1]-gaussian_radius):min(w, p[1]+gaussian_radius+1)
        ] += gaussian_map
    density_map = density_map / (np.sum(density_map / num_gt))
    return density_map

In [ ]:
#set the root to the dataset you want to use
root = 'CVHajjDataSet/'

In [ ]:
Aljoharah_st_train = os.path.join(root,'Aljoharah_st/train','images')
Aljoharah_st_val = os.path.join(root,'Aljoharah_st/val','images')
Aljoharah_st_test = os.path.join(root,'Aljoharah_st/test','images')

Jamarat_exit_train = os.path.join(root,'Jamarat_exit/train','images')
Jamarat_exit_val = os.path.join(root,'Jamarat_exit/val','images')
Jamarat_exit_test = os.path.join(root,'Jamarat_exit/test','images')

Jamarat_exit_with_kheif_train = os.path.join(root,'Jamarat_exit_with_kheif/train','images')
Jamarat_exit_with_kheif_val = os.path.join(root,'Jamarat_exit_with_kheif/val','images')
Jamarat_exit_with_kheif_test = os.path.join(root,'Jamarat_exit_with_kheif/test','images')

Tawaf_tabaeud_train = os.path.join(root,'Tawaf_tabaeud/train','images')
Tawaf_tabaeud_val = os.path.join(root,'Tawaf_tabaeud/val','images')
Tawaf_tabaeud_test = os.path.join(root,'Tawaf_tabaeud/test','images')

path_sets = [Aljoharah_st_train, Aljoharah_st_val, Aljoharah_st_test, Jamarat_exit_train, Jamarat_exit_val, Jamarat_exit_test, Jamarat_exit_with_kheif_train, Jamarat_exit_with_kheif_val, Jamarat_exit_with_kheif_test, Tawaf_tabaeud_train, Tawaf_tabaeud_val, Tawaf_tabaeud_test]



In [ ]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

In [ ]:
for img_path in tqdm(img_paths):
    img_ori = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    pts = loadmat(img_path.replace('.jpg', '.mat').replace('images', 'ground_truth').replace('IMG_', 'GT_IMG_'))
    img = cv2.imread(img_path)
    sigma = 15
    k = np.zeros((img.shape[0], img.shape[1]))
    gt = pts["image_info"][0, 0][0, 0][0]
    for i in range(len(gt)):
        if int(gt[i][1]) < img.shape[0] and int(gt[i][0]) < img.shape[1]:
            k[int(gt[i][1]), int(gt[i][0])] = 1

    DM = gen_density_map_gaussian(k, gt, sigma=sigma)

    file_path = img_path.replace('.jpg', '.h5').replace('images', 'ground_truth')
    with h5py.File(file_path, 'w') as hf:
        hf['density'] = DM

100%|██████████| 1512/1512 [01:39<00:00, 15.23it/s]


In [ ]:
#now see a sample from ShanghaiA
plt.imshow(Image.open(img_paths[5]))

In [ ]:
gt_file = h5py.File(img_paths[5].replace('.jpg','.h5').replace('images','ground_truth'),'r')
groundtruth = np.asarray(gt_file['density'])
plt.imshow(groundtruth,cmap=CM.jet)

In [ ]:
np.sum(groundtruth)# don't mind this slight variation